# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_Rivers.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,305.40,72.0,40.0,1.54,US,1.720295e+09
1,1,sedalia,38.7045,-93.2283,303.11,48.0,0.0,0.00,US,1.720295e+09
2,2,nagqu,31.5000,92.0000,279.15,89.0,100.0,0.92,CN,1.720295e+09
3,3,puerto natales,-51.7236,-72.4875,269.40,100.0,100.0,0.00,CL,1.720295e+09
4,4,russkaya polyana,49.4170,31.9201,290.64,58.0,3.0,3.38,UA,1.720295e+09


In [7]:
# convert the temperature to celsius
# K - 273.15 to the column to convert temperatures from Kelvin to Celsius.
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,32.25,72.0,40.0,1.54,US,1.720295e+09
1,1,sedalia,38.7045,-93.2283,29.96,48.0,0.0,0.00,US,1.720295e+09
2,2,nagqu,31.5000,92.0000,6.00,89.0,100.0,0.92,CN,1.720295e+09
3,3,puerto natales,-51.7236,-72.4875,-3.75,100.0,100.0,0.00,CL,1.720295e+09
4,4,russkaya polyana,49.4170,31.9201,17.49,58.0,3.0,3.38,UA,1.720295e+09


In [9]:
city_data_df['Date'] = pd.to_datetime(city_data_df['Date'], unit='s')
city_data_df['Date'] = city_data_df['Date'].dt.date
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,32.25,72.0,40.0,1.54,US,2024-07-06
1,1,sedalia,38.7045,-93.2283,29.96,48.0,0.0,0.00,US,2024-07-06
2,2,nagqu,31.5000,92.0000,6.00,89.0,100.0,0.92,CN,2024-07-06
3,3,puerto natales,-51.7236,-72.4875,-3.75,100.0,100.0,0.00,CL,2024-07-06
4,4,russkaya polyana,49.4170,31.9201,17.49,58.0,3.0,3.38,UA,2024-07-06


In [11]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     599 non-null    int64  
 1   City        599 non-null    object 
 2   Lat         576 non-null    float64
 3   Lng         576 non-null    float64
 4   Max Temp    576 non-null    float64
 5   Humidity    576 non-null    float64
 6   Cloudiness  576 non-null    float64
 7   Wind Speed  576 non-null    float64
 8   Country     569 non-null    object 
 9   Date        576 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 46.9+ KB


In [15]:
clean_df = city_data_df.dropna(how="any").reset_index(drop=True)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     569 non-null    int64  
 1   City        569 non-null    object 
 2   Lat         569 non-null    float64
 3   Lng         569 non-null    float64
 4   Max Temp    569 non-null    float64
 5   Humidity    569 non-null    float64
 6   Cloudiness  569 non-null    float64
 7   Wind Speed  569 non-null    float64
 8   Country     569 non-null    object 
 9   Date        569 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 44.6+ KB


---

### Step 1: Create a map that displays a point for every city in the `clean_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display

# Configure the map plot
map_plot_1 = clean_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 0.9,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 600
)

plt.savefig("map_plot_1.png")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

<Figure size 640x480 with 0 Axes>

### Step 2: Narrow down the `clean_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = clean_df.loc[
    (clean_df["Max Temp"] < 27) & (clean_df["Max Temp"] > 21) \
    & (clean_df["Wind Speed"] < 4.5) \
    & (clean_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,44,aral,40.5156,81.2636,25.23,34.0,0.0,2.70,CN,2024-07-06
92,96,oral,51.2333,51.3667,24.99,61.0,0.0,3.00,KZ,2024-07-06
105,110,al mughraqah,31.4659,34.4121,26.21,71.0,0.0,2.22,PS,2024-07-06
126,135,pisco,-13.7000,-76.2167,21.03,73.0,0.0,3.09,PE,2024-07-06
153,163,astrakhan,46.3497,48.0408,25.93,61.0,0.0,1.00,RU,2024-07-06


### Step 3: Create a new DataFrame called `hotel_df`.

In [63]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
43,aral,CN,40.5156,81.2636,34.0,
92,oral,KZ,51.2333,51.3667,61.0,
105,al mughraqah,PS,31.4659,34.4121,71.0,
126,pisco,PE,-13.7000,-76.2167,73.0,
153,astrakhan,RU,46.3497,48.0408,61.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.Lat
    longitude = row.Lng
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aral - nearest hotel: No hotel found
oral - nearest hotel: Гостиница Вокзала
al mughraqah - nearest hotel: منزل السيد/ اسامة جبر كحيل
pisco - nearest hotel: La Portada
astrakhan - nearest hotel: Omega
alghero - nearest hotel: Catalunya
buka - nearest hotel: No hotel found
vico del gargano - nearest hotel: Pineta
kintinku - nearest hotel: No hotel found
beira - nearest hotel: Hotel Savoy
prince george - nearest hotel: Travelodge by Wyndham Prince George
aladag - nearest hotel: No hotel found
cide - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
suva reka - nearest hotel: Hotel Solid
cesme - nearest hotel: Otel Tu Çeşme
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
43,aral,CN,40.5156,81.2636,34.0,No hotel found
92,oral,KZ,51.2333,51.3667,61.0,Гостиница Вокзала
105,al mughraqah,PS,31.4659,34.4121,71.0,منزل السيد/ اسامة جبر كحيل
126,pisco,PE,-13.7000,-76.2167,73.0,La Portada
153,astrakhan,RU,46.3497,48.0408,61.0,Omega
197,alghero,IT,40.5589,8.3181,78.0,Catalunya
269,buka,UZ,40.8108,69.1986,27.0,No hotel found
347,vico del gargano,IT,41.8983,15.9576,61.0,Pineta
357,kintinku,TZ,-5.8833,35.2333,52.0,No hotel found
360,beira,MZ,-19.8436,34.8389,83.0,Hotel Savoy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [79]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 0.9,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 400,
    hover_cols = ["Hotel Name", "Country"]
)

plt.savefig("map_plot_2.png")

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

<Figure size 640x480 with 0 Axes>